In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
# read trial-level IAT data (basefile) and subject-level experiment data (controlfile)
basefile = pd.read_csv('iat_conf_raw.csv', delimiter=',', index_col=False)
controlfile = pd.read_csv('ideology_confirmatory.csv', delimiter=',', index_col=False)

C:\Users\kjl113\AppData\Local\Temp\ipykernel_2900\2062624857.py:3: DtypeWarning: Columns (36,44,46,49,50,51,52,53,56,69,70,71,72,73,74,75,76,86) have mixed types. Specify dtype option on import or set low_memory=False.
  controlfile = pd.read_csv('ideology_confirmatory.csv', delimiter=',', index_col=False)


In [3]:
df_dict = {}

# make a dictionary of trial-level IAT data with separate keys for each IAT
# first add the good/bad standard IATs (note that not all are included in the loop because not all task names map onto desired key names, e.g. typo in riskycautious in raw data)
for x in ['possiblecertain', 'labormanagement', 'statechurch', 'socialismcapitalism', 'complexsimple', 'protestaccept',
          'fairbiased', 'democracyfascism', 'motherfather', 'hopeduty', 'selfother', 'regulationmarkets',
          'progressrestore', 'newold', 'non-profitscorporations', '20501950', 'ambiguousclear',
          'defendattack', 'changepreserve', 'collectiveindividual', 'communityindividual', 'individualgroup',
          'forwardbackward', 'equalunequal', 'anarchyhierarchy', 'justiceinjustice',
          'novelfamiliar', 'foreignlocal', 'nurturingstrict', 'presentpast', ]:
    df_dict[x] = basefile[basefile['task_name'].isin(['i%s_gba' % x, 'i%s_gbb' % x])]

# next add all the true/false standard IATs
for x in ['dangersafety_tf', 'sciencereligion_tf', 'conspiracyaccident_tf',
          'evolutioncreationism_tf']:
    df_dict[x] = basefile[basefile['task_name'].isin(['i%sa' % x, 'i%sb' % x])]

# add any remaining IATs (those with long names, typos, the only danger/safe IAT, and the four single-target IATs)
df_dict['blackwhite'] = basefile[basefile['task_name'].isin(['iblackpeoplewhitepeople_gba', 'iwhitepeopleblackpeople_gbb'])]
df_dict['gaystraight'] = basefile[basefile['task_name'].isin(['istraightpeoplegaypeople_gba', 'istraightpeoplegaypeople_gbb'])]
df_dict['riskycautious'] = basefile[basefile['task_name'].isin(['irickycautious_gba', 'irickycautious_gbb'])]
df_dict['futurepresent'] = basefile[basefile['task_name'].isin(['ifurturepresent_gba', 'ifurturepresent_gbb'])]
df_dict['20501950_ds'] = basefile[basefile['task_name'].isin(['i20501950_dsa', 'i20501950_dsb'])]

# make a list of IATs that we want to invert their explicit preference scores, so they're in line with testing what is defined as compatible according to Ideology codebook 
invertlist = ['20501950', 'changepreserve', 'collectiveindividual', 'communityindividual',
              'defendattack', 'democracyfascism', 'equalunequal', 'evolutioncreationism_tf',
              'fairbiased', 'forwardbackward', 'hopeduty', 'individualgroup', 'justiceinjustice',
              'motherfather', 'newold', 'non-profitscorporations', 'nurturingstrict',
              'presentpast', 'progressrestore', 'sciencereligion_tf', 'selfother']

# make a dictionary with keys for each IAT, and all RDM thresholds for "Action B". Four different thresholds for four different blocks.
accumulator_b_dict = {'20501950_ds': ['2050', 'Safety', '2050/Safety', '1950/Safety'],
                   '20501950': ['1950', 'Bad', '1950/Bad', '2050/Bad'],
                   'ambiguousclear': ['Ambiguous', 'Bad', 'Ambiguous/Bad', 'Clear/Bad'],
                   'anarchyhierarchy': ['Anarchy', 'Bad', 'Anarchy/Bad', 'Hierarchy/Bad'],
                   'changepreserve': ['Preserve', 'Bad', 'Preserve/Bad', 'Change/Bad'],
                   'communityindividual': ['Individual', 'Bad', 'Individual/Bad', 'Community/Bad'],
                  'collectiveindividual': ['Individual', 'Bad', 'Individual/Bad', 'Collective/Bad'],
                  'conspiracyaccident_tf': ['Conspiracy', 'False', 'Conspiracy/False', 'Accident/False'],
                  'defendattack': ['Attack', 'Bad', 'Attack/Bad', 'Defend/Bad'],
                  'equalunequal': ['Unequal', 'Bad', 'Unequal/Bad', 'Equal/Bad'],
                  'evolutioncreationism_tf': ['Creationism', 'False', 'Creationism/False', 'Evolution/False'],
                  'foreignlocal': ['Foreign', 'Bad', 'Foreign/Bad', 'Local/Bad'],
                  'forwardbackward': ['Backward', 'Bad', 'Backward/Bad', 'Forward/Bad'],
                  'futurepresent': ['Future', 'Bad', 'Future/Bad', 'Present/Bad'],
                  'hopeduty': ['Duty', 'Bad', 'Duty/Bad', 'Hope/Bad'],
                  'individualgroup': ['Group', 'Bad', 'Group/Bad', 'Individual/Bad'],
                  'justiceinjustice': ['Injustice', 'Bad', 'Injustice/Bad', 'Justice/Bad'],
                  'motherfather': ['Father', 'Bad', 'Father/Bad', 'Mother/Bad'],
                  'newold': ['Old', 'Bad', 'Old/Bad', 'New/Bad'],
                  'non-profitscorporations': ['Corporations', 'Bad', 'Corporations/Bad', 'Non-Profits/Bad'],
                  'novelfamiliar': ['Novel', 'Bad', 'Novel/Bad', 'Familiar/Bad'],
                  'nurturingstrict': ['Strict', 'Bad', 'Strict/Bad', 'Nurturing/Bad'],
                  'presentpast': ['Past', 'Bad', 'Past/Bad', 'Present/Bad'],
                  'progressrestore': ['Restore', 'Bad', 'Restore/Bad', 'Progress/Bad'],
                  'regulationmarkets': ['Regulation', 'Bad', 'Regulation/Bad', 'Markets/Bad'],
                  'sciencereligion_tf': ['Religion', 'False', 'Religion/False', 'Science/False'],
                  'blackwhite': ['Black People', 'Bad', 'Black People/Bad', 'White People/Bad'],
                  'gaystraight': ['Gay People', 'Bad', 'Gay People/Bad', 'Straight People/Bad'],
                  'riskycautious': ['Risky', 'Bad', 'Risky/Bad', 'Cautious/Bad'],
                  'possiblecertain': ['Possible', 'Bad', 'Possible/Bad', 'Certain/Bad'],
                  'labormanagement': ['Labor', 'Bad', 'Labor/Bad', 'Management/Bad'],
                  'statechurch': ['State', 'Bad', 'State/Bad', 'Church/Bad'],
                  'socialismcapitalism': ['Socialism', 'Bad', 'Socialism/Bad', 'Capitalism/Bad'],
                  'complexsimple': ['Complex', 'Bad', 'Complex/Bad', 'Simple/Bad'],
                  'protestaccept': ['Protest', 'Bad', 'Protest/Bad', 'Accept/Bad'],
                  'fairbiased': ['Biased', 'Bad', 'Biased/Bad', 'Fair/Bad'],
                  'democracyfascism': ['Fascism', 'Bad', 'Fascism/Bad', 'Democracy/Bad'],
                  'dangersafety_tf': ['Danger', 'False', 'Danger/False', 'Safety/False'],
                  'selfother': ['Other', 'Bad', 'Other/Bad', 'Self/Bad']}

# make a dictionary of blocks with keys for each IAT. Blocks are recorded as integers for Stan to use as indices. 
# concept-only block = 1, attribute-only block = 2, compatible block = 3, incompatible block = 4
block_dict = {}
for y in list(df_dict.keys()):
      tempdict = {}
      x = accumulator_b_dict[y]
      tempdict['%s,%s' % (x[0],x[3].split('/')[0])] = 1
      tempdict['%s,%s' % (x[3].split('/')[0],x[0])] = 1
      if x[1] == 'Bad':
            tempdict['Good,Bad'] = 2
            tempdict['%s/Good,%s' % (x[3].split('/')[0], x[2])] = 3
            tempdict['%s/Good,%s' % (x[0], x[3])] = 4
      elif x[1] == 'False':
            tempdict['True,False'] = 2
            tempdict['%s/True,%s' % (x[3].split('/')[0], x[2])] = 3
            tempdict['%s/True,%s' % (x[0], x[3])] = 4
      elif x[1] == 'Safety':
            tempdict['Danger,Safety'] = 2
            tempdict['%s/Danger,%s' % (x[3].split('/')[0], x[2])] = 3
            tempdict['%s/Danger,%s' % (x[0], x[3])] = 4
      block_dict[y] = tempdict

In [4]:
def analyze(iat='blackwhite', save=False):

    df = df_dict[iat].copy() # get trial-level data for selected IAT
    accumulator_b = accumulator_b_dict[iat] # get list of labels for "Action B", the second accumulator in each of the four blocks

    # df = df[df['trial_error'] == 0] # limit the data to only correct responses (error trials have confounded RTs, and we're instead interested in evidence accumulation toward either action when correct)
    df = df[['session_id', 'trial_response', 'trial_latency', 'trial_error', 'block_number', 'block_pairing_definition']] # limit data to what we need for Stan (ID, action, RT, and block)
    df = df.dropna(axis=0).reset_index(drop=True) # of these limited data, drop any trials that are missing the information needed for Stan

    # convert the nominal variables into integers for Stan
    df['trial_response'] = [1 if x in accumulator_b else 2 for x in df['trial_response']]
    df['block_pairing_definition'] = [block_dict[iat][x] for x in df['block_pairing_definition']]

    print('Number of original subjects: %s' % len(df['session_id'].unique()))

    # remove any subjects that lack minimal variability in their actions within blocks; this is mostly due to incomplete sessions / missing blocks
    dellist = []
    for x in df['session_id'].unique():
        for y in df['block_pairing_definition'].unique():
            temp = df[df['session_id']==x]
            if list(temp[temp['block_pairing_definition']==y]['trial_response']).count(1) < 2:
                dellist.append(x)
            elif list(temp[temp['block_pairing_definition']==y]['trial_response']).count(2) < 2:
                dellist.append(x)
    print('Number of subjects thrown out due to no variation OR missing entire blocks: %s' % len(np.unique(dellist)))
    df = df[~df['session_id'].isin(dellist)]   

    print('Number of subjects remaining: %s' % len(df['session_id'].unique()))
    subs = np.unique(df['session_id']) # get a list of unique subjects

    # make lists for experiment data
    dlist = [[], [], [], [], [], []] # d-scores will go here
    session_id_list = [] # list of subjects who will get subject-level estimates in Stan
    session_id_list_ind = [] # index for each subject who will get subject-level estimates in Stan
    session_id_list_counter = 1 
    explist = [] # positive values are in favor of x

    df = df.reset_index(drop=True)
    d1df = df.copy()
    d1df = d1df[d1df['trial_latency'] <= 10000].reset_index(drop=True)
    d2df = df.copy()
    d2df = d2df[d2df['trial_latency'] <= 10000].reset_index(drop=True)
    d2df = d2df[d2df['trial_latency'] >= 400].reset_index(drop=True)
    d3df = df.copy()
    d3df = d3df[d3df['trial_latency'] <= 10000].reset_index(drop=True)
    d4df = d3df.copy()
    d5df = d3df[d3df['trial_latency'] >= 400].reset_index(drop=True)
    d6df = d5df.copy()
    in_correct_mean = np.mean(d3df[d3df['trial_error']==0].loc[d3df['block_pairing_definition']==4]['trial_latency'])
    c_correct_mean = np.mean(d3df[d3df['trial_error']==0].loc[d3df['block_pairing_definition']==3]['trial_latency'])
    in_correct_sd = np.std(d3df[d3df['trial_error']==0].loc[d3df['block_pairing_definition']==4]['trial_latency'])
    c_correct_sd = np.std(d3df[d3df['trial_error']==0].loc[d3df['block_pairing_definition']==3]['trial_latency'])
    in_correct_mean_56 = np.mean(d5df[d5df['trial_error']==0].loc[d5df['block_pairing_definition']==4]['trial_latency'])
    c_correct_mean_56 = np.mean(d5df[d5df['trial_error']==0].loc[d5df['block_pairing_definition']==3]['trial_latency'])
    in_correct_sd_56 = np.std(d5df[d5df['trial_error']==0].loc[d5df['block_pairing_definition']==4]['trial_latency'])
    c_correct_sd_56 = np.std(d5df[d5df['trial_error']==0].loc[d5df['block_pairing_definition']==3]['trial_latency'])
    for i, x in enumerate(d3df['trial_latency']):
        if d3df['trial_error'][i]==1 and d3df['block_pairing_definition'][i]==4:
            d3df['trial_latency'][i] = in_correct_mean + 2 * in_correct_sd
            d4df['trial_latency'][i] = in_correct_mean + 600
        elif d3df['trial_error'][i]==1 and d3df['block_pairing_definition'][i]==3:
            d3df['trial_latency'][i] = c_correct_mean + 2 * c_correct_sd
            d4df['trial_latency'][i] = c_correct_mean + 600
    for i, x in enumerate(d5df['trial_latency']):
        if d5df['trial_error'][i]==1 and d5df['block_pairing_definition'][i]==4:
            d5df['trial_latency'][i] = in_correct_mean_56 + 2 * in_correct_sd_56
            d6df['trial_latency'][i] = in_correct_mean_56 + 600
        elif d5df['trial_error'][i]==1 and d5df['block_pairing_definition'][i]==3:
            d5df['trial_latency'][i] = c_correct_mean_56 + 2 * c_correct_sd_56
            d6df['trial_latency'][i] = c_correct_mean_56 + 600

    # populate the above lists by looping through each subject
    for x in subs:
        rdmlen = len(df.loc[(df['session_id']==x) & (df['trial_error']==0) & (df['trial_latency']>200) & (df['trial_latency']<5000)])
        if rdmlen > 0:
            temp = controlfile[controlfile['session_id']==x].reset_index() # get experimental data for the subject
            if any(iat in task for task in [str(temp['task_%s' % t][0]) for t in [1, 2, 3, 4, 5, 6, 7, 8]]): # if the IAT was performed by the participant, and this is reflected in the experiment-level data
                for i, dfi in enumerate([d1df, d2df, d3df, d4df, d5df, d6df]):
                    incompatible = list(dfi[dfi['session_id']==x].loc[dfi['block_pairing_definition']==4]['trial_latency']) # get RTs for the subjects incompatible block
                    compatible = list(dfi[dfi['session_id']==x].loc[dfi['block_pairing_definition']==3]['trial_latency']) # get RTs for the subjects compatible block
                    dlist[i].append((np.mean(incompatible) - np.mean(compatible)) / np.std(incompatible + compatible)) # calculate D-score and append to list
            else:
                for i, dfi in enumerate([d1df, d2df, d3df, d4df, d5df, d6df]):
                    dlist[i].append(np.nan) # if no IAT listed in experimental data, append nan for this participant (they'll be excluded from Stan)
            if type(temp['explicit_task_full'][0]) == str: # if their experimental data reflects that an explicit preference thermometer was performed
                if all(x in temp['explicit_task_full'][0] for x in iat.split('_')): # and the IAT topic in question is what was evaluated
                    if iat in invertlist: # get explicit preference score and append to list. Invert the score if in the invertlist so they're in line with testing what is defined as compatible according to Ideology codebook
                        explist.append(temp['exp_att'][0])
                    else:
                        explist.append(temp['exp_att'][0]*(-1))
                else:
                    explist.append(np.nan)
            else:
                explist.append(np.nan) 
            if not np.isnan(dlist[0][-1]) and not np.isnan(explist[-1]):
                session_id_list_ind.append(session_id_list_counter) # append an index for the participant
                session_id_list.append(1) # append 1 to the id list so we can identify that this subject out of the entire pool will recieve subject-specific estimates
                session_id_list_counter += 1
            else:
                session_id_list.append(0)
                session_id_list_ind.append(0)

    print(len(dlist[0]))
    print(len(explist))
    standf = pd.DataFrame({'d': dlist[0], 'd2': dlist[1], 'd3': dlist[2], 'd4': dlist[3], 'd5': dlist[4],
                           'd6': dlist[5], 'exp': explist}).dropna(subset=['d', 'exp'])
    print('Number of subjects with d-score AND explicit preference AND selected for subject-specific estimates: %s' % len(standf))

    df = df[df['trial_error'] == 0] # limit the data to only correct responses (error trials have confounded RTs, and we're instead interested in evidence accumulation toward either action when correct)
    df = df[df['trial_latency'] > 200]
    df = df[df['trial_latency'] < 5000]

    # loop through every trial in the trial-level df, and assign an index to each unique subject. These indices will be used by Stan
    sub_ids = list(df['session_id'])
    new_inds = []
    counter = 1; trialcounter = 0
    new_d = []; new_exp = []; trialsper = []
    try:
        d = standf[standf['id']==sub_ids[0]].reset_index()['d'][0]
        exp = standf[standf['id']==sub_ids[0]].reset_index()['exp'][0]
    except:
        d = np.nan; exp = np.nan

    for i, x in enumerate(sub_ids):
        trialcounter += 1
        if i != 0:
            if x != sub_ids[i-1]:
                trialsper.append(trialcounter)
                trialcounter = 0
                counter += 1
                try:
                    d = standf[standf['id']==x].reset_index()['d'][0]
                    exp = standf[standf['id']==x].reset_index()['exp'][0]
                except:
                    d = np.nan; exp = np.nan
        new_inds.append(counter)
        new_d.append(d)
        new_exp.append(exp)

    # final dictionary for Stan data block
    data = {
        'N': len(df['session_id'].unique()), # number of unique subjects (everyone who completed the IAT)
        'T': len(df), # total number of observations
        'N_sub': np.sum(session_id_list), # total number of subjects who will get subject-level estimates (~10% for exploratory, ~50% for confirmatory)
        'N_ind': new_inds, # indices for each unique subject
        'N_sub_ind': session_id_list_ind, # indices for each subject who will get subject-level estimates
        'N_cond': 4, # total number of conditions / blocks
        'grainsize': int(round(len(df)) / 4), # grainsize for Stan to parallelize log-likelihood calculations. Not necessary.
        'condition': list(df['block_pairing_definition']), # conditions for each observation
        'RT': list(df['trial_latency']), # rts for each observation
        'choice': list(df['trial_response']), # choice for each observation
        'sub_id': sub_ids # participant ID
    }

    print(len(standf))
    print(data['N_sub'])

    if save == True:
        standf.to_csv('data/%s.csv' % iat, index=False) # save a csv with all the d-scores and explicit preferences
        with open("%s.pkl" % iat, "wb") as f:
            pickle.dump(data, f, protocol=-1) # save a pickle with the trial-level data for Stan
    
    return data['T'], np.mean(trialsper)

In [5]:
# run this for each IAT
trialslist = []
trialsperlist = []
for x in list(df_dict.keys()):
    trials, trialsper = analyze(x, save=False)
    trialslist.append(trials)
    trialsperlist.append(trialsper)


Number of original subjects: 3628
Number of subjects thrown out due to no variation OR missing entire blocks: 709
Number of subjects remaining: 2919


C:\Users\kjl113\AppData\Local\Temp\ipykernel_2900\857148806.py:59: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  d3df['trial_latency'][i] = in_correct_mean + 2 * in_correct_sd
C:\Users\kjl113\AppData\Local\Temp\ipykernel_2900\857148806.py:59

2919
2919
Number of subjects with d-score AND explicit preference AND selected for subject-specific estimates: 1853
1853
1853
Number of original subjects: 3463
Number of subjects thrown out due to no variation OR missing entire blocks: 583
Number of subjects remaining: 2880


C:\Users\kjl113\AppData\Local\Temp\ipykernel_2900\857148806.py:59: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  d3df['trial_latency'][i] = in_correct_mean + 2 * in_correct_sd
C:\Users\kjl113\AppData\Local\Temp\ipykernel_2900\857148806.py:59

2880
2880
Number of subjects with d-score AND explicit preference AND selected for subject-specific estimates: 1924
1924
1924
Number of original subjects: 3645
Number of subjects thrown out due to no variation OR missing entire blocks: 609
Number of subjects remaining: 3036


...

2848
2848
Number of subjects with d-score AND explicit preference AND selected for subject-specific estimates: 1678
1678
1678
Number of original subjects: 3580
Number of subjects thrown out due to no variation OR missing entire blocks: 634
Number of subjects remaining: 2946


C:\Users\kjl113\AppData\Local\Temp\ipykernel_2900\857148806.py:59: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  d3df['trial_latency'][i] = in_correct_mean + 2 * in_correct_sd
C:\Users\kjl113\AppData\Local\Temp\ipykernel_2900\857148806.py:59

2946
2946
Number of subjects with d-score AND explicit preference AND selected for subject-specific estimates: 2024
2024
2024
Number of original subjects: 3606
Number of subjects thrown out due to no variation OR missing entire blocks: 624
Number of subjects remaining: 2982


C:\Users\kjl113\AppData\Local\Temp\ipykernel_2900\857148806.py:59: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  d3df['trial_latency'][i] = in_correct_mean + 2 * in_correct_sd
C:\Users\kjl113\AppData\Local\Temp\ipykernel_2900\857148806.py:59

2982
2982
Number of subjects with d-score AND explicit preference AND selected for subject-specific estimates: 2431
2431
2431


In [12]:
np.std(trialslist)

24651.21206795846

In [14]:
np.std(trialsperlist)

2.8281687784490215